In [ ]:
%%time

import neo4j
import csv
import pandas as pd

driver = neo4j.GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "1234")) 
driver.verify_connectivity()

In [ ]:
def run_query(driver, query):
    with driver.session() as session:
        # result = session.run(query)
        result = session.run(query)
        if result != None: 
            return result.value()
        else:
            return None

def aggregate_activities(driver):
    results = []
    with driver.session() as session:
        result = session.run(f"""
        MATCH (e:Event) WITH distinct e.activity AS act_name
        CALL {{
            WITH act_name
            MERGE (c:Class {{Name: act_name, Type: "Activity", ID: act_name}})
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
        result = session.run(f"""
        MATCH (c:Class) WHERE c.Type = "Activity"
        MATCH (e:Event) WHERE c.Name = e.activity
        CALL {{
            WITH c, e
            MERGE (e)-[:OBSERVED]->(c)
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
    return results

def lift_activity_df_relations(driver):
    with driver.session() as session:
        result = session.run(f"""        
        CALL {{
            MATCH (c1: Class)<-[:OBSERVED]-(e1: Event)-[df:DF_ENTITY]->(e2: Event)-[:OBSERVED]->(c2: Class)
            MATCH (e1)-[:CORR]->(n)<-[:CORR]-(e2) WHERE c1.Type = c2.Type AND n.EntityType = df.EntityType
            WITH n.EntityType as EType, c1, count(df) AS df_freq, c2 WHERE df_freq > 100
            MERGE (c1)-[rel: DF_C {{EntityType: EType}}]->(c2) ON CREATE SET rel.count = df_freq
        }} IN TRANSACTIONS
        """)
    return [result.consume()]

def aggregate_activity_entity(driver):
    results = []
    with driver.session() as session:
        result = session.run(f"""
        CALL {{
            MATCH (e:Event)-[:CORR]->(n:Entity) WITH distinct e.activity as actName,n.EntityType as EType
            WITH actName, EType
            MERGE (c: Class {{ID: actName+"_"+EType, Name:actName, EntityType:EType, Type:"Activity,EntityType"}})
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
        result = session.run(f"""
        CALL {{
            MATCH (c: Class) WHERE c.Type = "Activity,EntityType"
            WITH c
            MATCH (e:Event)-[:CORR]->(n:Entity) WHERE c.Name = e.activity AND c.EntityType=n.EntityType
            MERGE (e)-[:OBSERVED]->(c)
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
    return results

def lift_dfs(driver):
    with driver.session() as session:
        result = session.run(f"""
        CALL {{
        MATCH (c1: Class)<-[:OBSERVED]-(e1:Event)-[df:DF_ENTITY]->(e2:Event)-[:OBSERVED]->(c2:Class)
        MATCH (e1)-[:CORR]->(n)<-[:CORR]-(e2)
        WHERE c1.Type = c2.Type AND n.EntityType = df.EntityType AND c1.EntityType=n.EntityType AND c2.EntityType=n.EntityType
        WITH n.EntityType as EType, c1, count(df) AS df_freq, c2
        MERGE (c1)-[rel2:DF_C {{EntityType:EType}}]->(c2) ON CREATE SET rel2.count=df_freq
        }} IN TRANSACTIONS
        """)
    return [result.consume()]

def add_synchronisation(driver):
    with driver.session() as session:
        result = session.run(f"""
            CALL {{
                MATCH (c1:Class), (c2:Class) WHERE c1.Name=c2.Name AND c1.EntityType <> c2.EntityType
                MERGE (c1)-[:SYNC]->(c2)
            }} IN TRANSACTIONS
        """)
    return [result.consume()]

def delete_df_activities(driver):
    results = []
    with driver.session() as session:
        result = session.run(f"""
        MATCH (c1:Class)-[df:DF_C]->(c2:Class)
        CALL {{
            WITH df
            DELETE df
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
        result = session.run(f"""
        MATCH (e:Event)-[obs:OBSERVED]->(c:Class)
        CALL {{
            WITH obs
            DELETE obs
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
        result = session.run(f"""
        MATCH (:Class)-[r]->()
        CALL {{
            WITH r
            DELETE r
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
        results.append(result.consume())
        result = session.run(f"""
        MATCH ()-[r]->(:Class)
        CALL {{
            WITH r
            DELETE r
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
        result = session.run(f"""
        MATCH (c:Class)
        CALL {{
            WITH c
            DELETE c
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
    return results
    
def delete_aggregations(driver):
    results = []
    with driver.session() as session:
        result = session.run(f"""
            MATCH (c1)-[df:DF_C]->(c2) WHERE c1.Type="Activity,EntityType" OR c2.Type="Activity,EntityType"
            CALL {{
                WITH df
                DELETE df
            }} IN TRANSACTIONS
            """)
        results.append(result.consume())
        result = session.run(f"""
        MATCH (e)-[obs:OBSERVED]->(c) WHERE c.Type="Activity,EntityType"
        CALL {{
            WITH obs
            DELETE obs
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
        result = session.run(f"""
        MATCH (c:Class) WHERE c.Type="Activity,EntityType"
        CALL {{
            WITH c
            DELETE c
        }} IN TRANSACTIONS
        """)
        results.append(result.consume())
    return results

In [ ]:
# %%time
# delete_df_activities(driver)
# delete_aggregations(driver)

In [ ]:
%%time
aggregate_activities(driver)

In [ ]:
%%time
lift_activity_df_relations(driver)

In [ ]:
%%time
aggregate_activity_entity(driver)

In [ ]:
%%time
lift_dfs(driver)

In [ ]:
%%time
add_synchronisation(driver)